# Метод Ridge с поиском лучших параметров через GridSearchCV

In [1]:
# Импортируем необходимые библиотеки

import numpy as np                                      # для работы с массивами данных
import pandas as pd                                     # для обработки и анализа данных (чтобы красиво смотрелись таблички)
import seaborn as sns                                   # для визуализации
import xlrd                                             # для чтения Excel-файлов

from sklearn.preprocessing import StandardScaler        # для стандартизации данных (приведение к нормальному распределению)

from sklearn.model_selection import train_test_split    # для разделения на тестовую о обучающую выборки
from sklearn.metrics import r2_score

from sklearn.linear_model import Ridge                  # алгоритм машинного обучения
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt                         # для визуализации
%matplotlib inline

In [2]:
dataset = pd.read_excel('ebw_data.xlsx')                # загружаем датасет
dataset.head()                                          # смотрим первые 5 значений сверху

,IW,IF,VW,FP,Depth,Width
0,47,139,4.5,80,1.60,2.54
1,47,139,4.5,80,1.62,2.50
2,47,139,4.5,80,1.68,2.60
3,47,139,4.5,80,1.58,2.52
4,45,140,4.5,80,1.76,2.48


In [3]:
std_scaler = StandardScaler()                                                         # вызываем метод
dataset_std = std_scaler.fit_transform(np.array(dataset[['IW', 'IF', 'VW', 'FP']]))   # обучаем нормализатор
dataset_std[:1]                                                                       # смотрим первую строчку

array([[ 0.8       , -0.45662965, -2.02221044,  0.07808688]])

In [4]:
# вернём из np.array обратно в DataFrame 
# в видеолекции "Нормализация и стандартизация, практика" Г.В. Соколов так делал, и мы тоже будем

dataset_std_df = pd.DataFrame(data = dataset_std, columns = ['IW', 'IF', 'VW', 'FP'])
dataset_std_df.head()

,IW,IF,VW,FP
0,0.8,-0.456630,-2.02221,0.078087
1,0.8,-0.456630,-2.02221,0.078087
2,0.8,-0.456630,-2.02221,0.078087
3,0.8,-0.456630,-2.02221,0.078087
4,-0.4,-0.260931,-2.02221,0.078087


In [5]:
# Зависимость глубины сварных швов от совокупности параметров технологических режимов 

y1_columns = ['Depth']                                                 # это наши выходные значения №1
x1_columns = ['IW', 'IF', 'VW', 'FP']                                  # это наши входные значения

y1 = dataset.loc[:, y1_columns]                                        # выходные значения берём из "оригинального" датасета
x1 = dataset_std_df.loc[:, x1_columns]                                 # входные значения берём из стандартизированного датасета

# разделяем выборку на тестовую и обучающую
# с параметрами поигрались, чтобы достичь нормальных показателей обучения

x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, 
                                                        test_size=0.25, 
                                                        shuffle = True, 
                                                        random_state = 15)

In [6]:
# Зависимость ширины сварных швов от совокупности параметров технологических режимов 

y2_columns = ['Width']                                                 # это наши выходные значения №2
x2_columns = ['IW', 'IF', 'VW', 'FP']

y2 = dataset.loc[:, y2_columns]
x2 = dataset_std_df.loc[:, x2_columns]

# разделяем выборку на тестовую и обучающую
# параметры модели оставим такие же, как в x1, y1

x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, 
                                                        test_size=0.25, 
                                                        shuffle = True, 
                                                        random_state = 15)

In [8]:
model1 = Ridge()
param_grid = {'alpha': np.arange(0, 2, 0.01, dtype=float),
             'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']}
grid = GridSearchCV(model1, param_grid = param_grid)
grid.fit(x1_train, y1_train)

print('****Результаты метода****')
y1_train_pred = grid.predict(x1_train)
y1_test_pred = grid.predict(x1_test)
    
R2_train = r2_score(y1_train, y1_train_pred)
R2_test = r2_score(y1_test, y1_test_pred)
    
print('R2 score для тренировочной выборки: ', R2_train, 
         '\nR2 score для тестовой выборки: ', R2_test)


****Результаты метода****
R2 score для тренировочной выборки:  0.8088101147892706 
R2 score для тестовой выборки:  0.7880287397056892


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1000 fits failed out of a total of 8000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1000 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 1011, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 705, in fit
    raise ValueError(
ValueError: 'lbfgs' s

In [9]:
print(grid.best_score_)
print(grid.best_estimator_)

0.6308606403146377
Ridge(alpha=0.0, solver='svd')


In [10]:
model2 = Ridge()
param_grid = {'alpha': np.arange(0, 2, 0.01, dtype=float),
             'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']}
grid = GridSearchCV(model2, param_grid = param_grid)
grid.fit(x2_train, y2_train)

print('****Результаты метода****')
y2_train_pred = grid.predict(x2_train)
y2_test_pred = grid.predict(x2_test)
    
R2_train = r2_score(y2_train, y2_train_pred)
R2_test = r2_score(y2_test, y2_test_pred)
    
print('R2 score для тренировочной выборки: ', R2_train, 
         '\nR2 score для тестовой выборки: ', R2_test)

****Результаты метода****
R2 score для тренировочной выборки:  0.9383180813262846 
R2 score для тестовой выборки:  0.9564039105534151


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1000 fits failed out of a total of 8000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1000 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 1011, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 705, in fit
    raise ValueError(
ValueError: 'lbfgs' s

In [11]:
print(grid.best_score_)
print(grid.best_estimator_)

0.8595746956504472
Ridge(alpha=1.69, solver='saga')


Отметим, что для нахождения Depth метод Ridge не подходит, т.к. лучшие значения достигаются при alpha=0.0.
А в документации по методу сказано, что в таком случае использование данного метода не рекомендуется.
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html